In [1]:
import os
import ast
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

# 禁止随机，结果可复现
random.seed(42)
np.random.seed(42)

## 超参数选择

In [2]:
time_ratio = [1/2, 3/4, 7/8, 1][3]

In [3]:
# Parameters
stage_id = 0
data_number = 200

In [4]:
stage = ['计算矩阵', '读取矩阵'][stage_id]
model = 'Deep-learning/Ours'
emo_index = ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'][0]


ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering'
Clustering_Method = 'DTW_KMeans'
Emotion_Data_PATH = f'{ROOT_PATH}/data/Emotion_Data/{model}'   # 情绪数据路径
Financial_Data_PATH = f'{ROOT_PATH}/data/Financial_Data' # 金融数据路径

print(f"Running with: emo_index={emo_index}, stage={stage}, model={model}, data_number={data_number}")

Running with: emo_index=上证综合情绪值, stage=计算矩阵, model=Deep-learning/Ours, data_number=200


## 数据准备

In [5]:
"""读取股吧个股的数据"""
all_data = []

file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

for file in tqdm(file_list, desc="读取文件"):
    file_path = os.path.join(Emotion_Data_PATH, file)
    try:
        df = pd.read_csv(file_path, usecols=['日期', '高维情绪变量'])  # 只读需要的列
    except Exception as e:
        print(f"读取失败 {file}: {e}")
        continue

    stock_code = os.path.splitext(file)[0]

    df['股票编号'] = stock_code
    all_data.append(df)

guba_data = pd.concat(all_data, ignore_index=True)

读取文件:   0%|          | 0/183 [00:00<?, ?it/s]

读取文件:   1%|          | 1/183 [00:00<01:21,  2.22it/s]

读取文件:   1%|          | 2/183 [00:00<00:48,  3.76it/s]

读取文件:   2%|▏         | 3/183 [00:00<00:55,  3.25it/s]

读取文件:   2%|▏         | 4/183 [00:02<02:04,  1.44it/s]

...

读取文件:  98%|█████████▊| 179/183 [01:08<00:01,  3.19it/s]

读取文件:  99%|█████████▉| 181/183 [01:08<00:00,  4.32it/s]

读取文件:  99%|█████████▉| 182/183 [01:09<00:00,  4.20it/s]

读取文件: 100%|██████████| 183/183 [01:10<00:00,  2.03it/s]

读取文件: 100%|██████████| 183/183 [01:10<00:00,  2.59it/s]

In [6]:
# 查看最早和最晚日期
earliest_date = guba_data["日期"].min()
latest_date = guba_data["日期"].max()

print("最早日期：", earliest_date)
print("最晚日期：", latest_date)


最早日期： 2021-01-01
最晚日期： 2024-12-31


In [7]:
guba_data = guba_data.sort_values(by="日期").reset_index(drop=True)
idx_ratio = int(len(guba_data) * time_ratio)  
guba_data = guba_data.iloc[:idx_ratio]
guba_data

,日期,高维情绪变量,股票编号
0,2021-01-01,"[-0.2253921627998352, 0.3089153468608856, 0.38...",601919
1,2021-01-01,"[0.08484992384910583, 0.4280889928340912, 0.46...",601012
2,2021-01-01,"[-0.23067481815814972, 0.23074224591255188, 0....",601318
3,2021-01-01,"[-0.16127993166446686, 0.09594926238059998, 0....",000625
4,2021-01-01,"[-0.22539249062538147, 0.308915376663208, 0.38...",000725
...,...,...,...
495326,2024-12-31,"[-0.1518116295337677, 0.012850590981543064, 0....",002594
495327,2024-12-31,"[-0.4032808542251587, -0.18436959385871887, 0....",000725
495328,2024-12-31,"[-0.0021896061953157187, -0.08728286623954773,...",300999
495329,2024-12-31,"[-0.2693434953689575, 0.10387402027845383, 0.5...",300676


In [8]:
# 查看最早和最晚日期
earliest_date = guba_data["日期"].min()
latest_date = guba_data["日期"].max()

print("最早日期：", earliest_date)
print("最晚日期：", latest_date)


最早日期： 2021-01-01
最晚日期： 2024-12-31


In [9]:
"""读取股票回报率的数据"""
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

,股票编号,日期,交易量,收益率变化
0,000002,2021-06-01,60990961,-0.003745
1,000002,2021-06-02,85354506,0.006015
2,000002,2021-06-03,50594187,-0.003363
3,000002,2021-06-04,71422364,-0.012748
4,000002,2021-06-07,64745280,-0.014812
...,...,...,...,...
154877,688981,2024-11-20,58507495,-0.017071
154878,688981,2024-11-21,56197106,0.002358
154879,688981,2024-11-22,79240108,-0.050588
154880,688981,2024-11-25,76905909,-0.029402


In [10]:
# 进行左连接，guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = merged_data.dropna()
merged_data

,日期,高维情绪变量,股票编号,交易量,收益率变化
6439,2021-06-01,"[-0.10334751009941101, -0.02684004232287407, 0...",601166,118168969.0,-0.022068
6440,2021-06-01,"[-0.12155835330486298, 0.17990195751190186, 0....",002241,119791643.0,0.034403
6441,2021-06-01,"[-0.406835675239563, 0.10095615684986115, 0.43...",601857,128459929.0,0.019481
6442,2021-06-01,"[-0.025604138150811195, -0.01249769888818264, ...",300896,2932326.0,0.004877
6443,2021-06-01,"[0.006676249671727419, 0.3031323552131653, 0.2...",300896,2932326.0,0.004877
...,...,...,...,...,...
495131,2024-11-22,"[-0.03479849919676781, 0.017065513879060745, 0...",300236,8053791.0,-0.045638
495132,2024-11-22,"[-0.369239866733551, 0.13421954214572906, 0.17...",000061,27670515.0,0.027221
495133,2024-11-22,"[-0.2279377579689026, 0.4055686295032501, 0.24...",600600,4378607.0,-0.029869
495134,2024-11-22,"[-0.0671573206782341, 0.01824379898607731, 0.2...",300236,8053791.0,-0.045638


## 数据预处理

In [11]:
# 对每个股票编号的数据进行标准化和按日期汇总
tqdm.pandas()

def process_data(df):
    df_processed = []

    # 确保高维情绪变量是 list 类型（如果还没转过）
    if isinstance(df['高维情绪变量'].iloc[0], str):
        df['高维情绪变量'] = df['高维情绪变量'].progress_apply(ast.literal_eval)

    # 对每个股票编号处理，加 tqdm 进度条
    for stock_code, stock_data in tqdm(df.groupby('股票编号'), desc="Processing stocks"):
        for date, group in stock_data.groupby('日期'):
            try:
                emotion_matrix = np.array(group['高维情绪变量'].tolist())  # (样本数, 维度)
                avg_emotion = emotion_matrix.mean(axis=0).round(3).tolist()

                summary_row = {
                    '股票编号': stock_code,
                    '日期': date,
                    '高维情绪变量': avg_emotion,
                    '交易量': group['交易量'].mean(),
                    '收益率变化': group['收益率变化'].mean()
                }
                df_processed.append(summary_row)
            except Exception as e:
                print(f"跳过股票{stock_code} 日期{date}：{e}")
                continue

    return pd.DataFrame(df_processed)
    
# 调用处理函数
final_data = process_data(merged_data)
final_data

  0%|          | 0/391899 [00:00<?, ?it/s]

  0%|          | 23/391899 [00:00<30:07, 216.82it/s]

  0%|          | 83/391899 [00:00<15:01, 434.49it/s]

  0%|          | 142/391899 [00:00<12:56, 504.22it/s]

  0%|          | 202/391899 [00:00<12:07, 538.62it/s]

...

Processing stocks:  98%|█████████▊| 179/183 [00:30<00:00,  8.62it/s]

Processing stocks:  99%|█████████▉| 181/183 [00:30<00:00,  9.86it/s]

Processing stocks: 100%|██████████| 183/183 [00:31<00:00,  9.18it/s]

Processing stocks: 100%|██████████| 183/183 [00:31<00:00,  5.89it/s]

,股票编号,日期,高维情绪变量,交易量,收益率变化
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",60990961.0,-0.003745
1,000002,2021-06-08,"[-0.276, -0.035, 0.247, 0.326, 0.018, 0.244, -...",44676494.0,0.004626
2,000002,2021-06-10,"[-0.083, 0.072, 0.273, 0.26, -0.087, 0.276, -0...",53800776.0,-0.010035
3,000002,2021-06-11,"[-0.114, 0.163, 0.381, 0.325, -0.091, 0.088, -...",75853738.0,-0.014035
4,000002,2021-06-15,"[-0.169, 0.147, 0.442, 0.245, -0.009, 0.368, -...",89915501.0,-0.020957
...,...,...,...,...,...
85724,688981,2024-11-12,"[-0.134, -0.011, 0.308, 0.208, 0.049, 0.263, -...",108866759.0,-0.036864
85725,688981,2024-11-13,"[-0.228, 0.028, 0.358, 0.294, -0.137, 0.178, -...",80759477.0,-0.019484
85726,688981,2024-11-14,"[-0.18, 0.054, 0.392, 0.292, -0.048, 0.215, -0...",76194102.0,-0.022897
85727,688981,2024-11-15,"[-0.159, 0.022, 0.377, 0.319, -0.086, 0.268, -...",71066743.0,-0.020233


## DTW 聚类

In [12]:
# 选择需要的列
dtw_df = final_data[['日期', '股票编号', '高维情绪变量', '收益率变化']]

# 按照股票编号和日期排序
dtw_df = dtw_df.sort_values(by=['股票编号', '日期'])

# 创建一个以股票编号为键，日期为时间序列的字典
stock_data = {}
if data_number > len(dtw_df['股票编号'].unique()):
    data_number = len(dtw_df['股票编号'].unique())
for stock in dtw_df['股票编号'].unique()[:data_number]:
    stock_data[stock] = dtw_df[dtw_df['股票编号'] == stock][['日期', '高维情绪变量', '收益率变化']].reset_index(drop=True)

In [13]:
if stage == '计算矩阵':
    from tqdm import tqdm
    from fastdtw import fastdtw
    from scipy.spatial.distance import euclidean
    
    # 假设stock_data已经准备好了，stock_ids包含所有股票编号
    stock_ids = list(stock_data.keys())
    dtw_distances = np.zeros((len(stock_ids), len(stock_ids)))
    
    # 使用 tqdm 包装外层循环以显示进度条
    for i, stock_i in tqdm(enumerate(stock_ids), total=len(stock_ids), desc="计算 DTW 距离", unit="股票"):
        for j, stock_j in enumerate(stock_ids):
            if i < j:
                # 构造 series_i（二维数组，每行是 [情绪1, ..., 情绪N, 收益率]）
                df_i = stock_data[stock_i]
                series_i = [
                    emotion_vec + [ret]
                    for emotion_vec, ret in zip(df_i['高维情绪变量'], df_i['收益率变化'])
                ]

                df_j = stock_data[stock_j]
                series_j = [
                    emotion_vec + [ret]
                    for emotion_vec, ret in zip(df_j['高维情绪变量'], df_j['收益率变化'])
                ]

                # 使用 fastdtw 计算 DTW 距离
                distance, _ = fastdtw(series_i, series_j, dist=euclidean)
                dtw_distances[i, j] = distance
                dtw_distances[j, i] = distance  # 对称填充

    # 保存结果
    save_dir = f'{ROOT_PATH}/{Clustering_Method}/个股分析/{data_number}/{model}/个股分析_高维情绪'
    os.makedirs(save_dir, exist_ok=True)
    np.save(f'{save_dir}/dtw_distances.npy', dtw_distances)
    print(f'DTW距离矩阵已保存至：{save_dir}')

计算 DTW 距离:   0%|          | 0/183 [00:00<?, ?股票/s]

计算 DTW 距离:   1%|          | 1/183 [00:32<1:37:10, 32.03s/股票]

计算 DTW 距离:   1%|          | 2/183 [00:47<1:08:04, 22.57s/股票]

计算 DTW 距离:   2%|▏         | 3/183 [01:17<1:16:47, 25.60s/股票]

计算 DTW 距离:   2%|▏         | 4/183 [01:36<1:09:10, 23.19s/股票]

...

计算 DTW 距离:  98%|█████████▊| 179/183 [33:52<00:03,  1.25股票/s]

计算 DTW 距离:  98%|█████████▊| 180/183 [33:52<00:01,  1.60股票/s]

计算 DTW 距离:  99%|█████████▉| 181/183 [33:52<00:00,  2.07股票/s]

计算 DTW 距离: 100%|██████████| 183/183 [33:52<00:00, 11.11s/股票]

DTW距离矩阵已保存至：/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering/DTW_KMeans/个股分析/183/Deep-learning/Ours/个股分析_高维情绪


In [14]:
# 读取 DTW 距离矩阵
stock_ids = list(stock_data.keys())
dtw_distances = np.load(f'{ROOT_PATH}/{Clustering_Method}/个股分析/{data_number}/{model}/个股分析_高维情绪/dtw_distances.npy')

In [15]:
from sklearn.cluster import KMeans

# KMeans 聚类
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(dtw_distances)

# 查看每个股票的聚类结果
stock_clusters = {stock_ids[i]: clusters[i] for i in range(len(stock_ids))}

# 创建一个 DataFrame 来保存股票编号与对应的聚类标签
stock_clusters_df = pd.DataFrame({
    '股票编号': stock_ids,
    '聚类标签': clusters
})

# 保存到 CSV
os.makedirs('output', exist_ok=True)
stock_clusters_df.to_csv(f'output/{time_ratio}.csv', index=False)

# 查看结果
stock_clusters_df


,股票编号,聚类标签
0,000002,2
1,000061,1
2,000063,2
3,000069,1
4,000100,2
...,...,...
178,603986,2
179,688005,1
180,688027,1
181,688029,1
